In [1]:
import numpy as np
import pandas as pd
import keras
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [2]:
properties = pd.read_csv('./data/properties_modifiedV2.csv').drop('Unnamed: 0', axis=1)
#transactions = pd.read_csv('./data/train_2016_v2.csv')

/home/matthew/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (23,33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Feature Cleaning

This data has undergone some engineering, but we'll still need to adjust it for neural net input.

In [4]:
parcels = properties['parcelid']
properties.drop(['propertyzoningdesc','regionidneighborhood','censustractandblock','rawcensustractandblock'], axis=1, inplace=True)

In [5]:
id_cols = [c for c in properties.columns if 'id' in c]

properties[id_cols] = properties[id_cols].apply(lambda x: x.astype('category'))
properties['propertycountylandusecode'] = properties['propertycountylandusecode'].astype('category')
properties['fips'] = properties['fips'].astype('category')
properties['yearbuilt'] = properties['yearbuilt'].fillna(9999)

In [ ]:
properties = properties.dropna(thresh=0.7*properties.shape[0], axis=1)

scaler = StandardScaler()

num_cols = [c for c in properties.columns if 'num' in c]
properties[num_cols] = properties[num_cols].fillna(-1)

q_cols = (properties.dtypes[properties.dtypes != 'category']).index
properties = properties.fillna(properties.mean())

properties[q_cols] = scaler.fit_transform(properties[q_cols])
properties = pd.get_dummies(properties, dummy_na=True)


/usr/local/lib/python3.5/dist-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


We'll fill the rest of the null values with the simple mean of the data

In [77]:
train = transactions.merge(properties, how='left', on='parcelid')

In [85]:
labels = train['logerror']
train.drop('logerror', axis=1, inplace=True)